In [82]:
import torch
import dill
import pytorch_lightning as pl
import transformers
import pandas as pd
from dataloader import Dataloader
import torchmetrics
from model import Model

In [83]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    "beomi/KcELECTRA-base", max_length=160
)
new_tokens = pd.read_csv("new_token.csv").columns.tolist()
#special_tokens_dict = {"additional_special_tokens": ["[RTT]", "[ORG]"]}
#tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.add_tokens(new_tokens)

model = Model(
        "beomi/KcELECTRA-base",
        2e-5,
        35,
        0.1,
        1,
        tokenizer,
    )
trainer = pl.Trainer(gpus=1, log_every_n_steps=1)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.bias', 'classifier

In [84]:
len(tokenizer)

50178

In [102]:
dataloader = Dataloader(
'beomi/KcELECTRA-base',
32,
False,
"../../data/train_cat_basic.csv",
"../../data/dev_rtt.csv",
"../../data/dev_rtt.csv",
"../../data/dev_rtt.csv",
4,
tokenizer,
)

In [97]:
checkpoint = torch.load(
        "STS/2otzps5c/checkpoints/epoch=19-step=2080.ckpt",
        pickle_module=dill,
    )

In [98]:
model.load_state_dict(
    checkpoint["state_dict"],
)

<All keys matched successfully>

In [99]:
result = trainer.test(model=model, datamodule=dataloader)

tokenizing: 100%|██████████| 197/197 [00:00<00:00, 2366.37it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 7/7 [00:02<00:00,  3.47it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_pearson          0.5015915632247925
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [89]:
predictions = trainer.predict(model=model, datamodule=dataloader)

tokenizing: 100%|██████████| 197/197 [00:00<00:00, 2407.09it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 7/7 [00:02<00:00,  3.41it/s]


In [90]:
predictions

[tensor([2.7184, 3.3949, 3.7171, 2.2153, 3.6620, 2.9940, 2.8440, 3.6777, 2.8628,
         3.4470, 3.5466, 3.0331, 3.0879, 2.4312, 3.2750, 3.2705, 2.9157, 4.1383,
         2.5754, 3.9930, 2.6795, 4.3111, 3.6825, 2.5970, 3.2171, 3.8972, 2.5063,
         3.6565, 3.7053, 3.0609, 3.6666, 2.5742]),
 tensor([2.7214, 3.2415, 3.0770, 2.8586, 3.6461, 3.1370, 3.4643, 3.2917, 3.8027,
         3.3294, 3.7863, 4.2594, 3.5519, 3.4564, 3.2899, 3.6118, 4.2223, 4.3231,
         4.1885, 4.4929, 2.7984, 3.1039, 3.1812, 3.5766, 3.9948, 3.8924, 3.9008,
         3.1690, 3.4872, 3.1314, 2.9220, 3.6606]),
 tensor([4.2995, 2.8775, 4.1832, 4.0051, 3.7776, 3.8638, 3.2725, 3.4666, 3.7297,
         2.3996, 2.7605, 3.0823, 3.2004, 3.8058, 3.0786, 2.6340, 3.8227, 3.5953,
         3.6699, 3.5709, 3.7608, 3.2864, 2.4395, 3.0635, 3.4158, 4.5634, 3.7425,
         3.2537, 1.3791, 3.9608, 3.1106, 3.7979]),
 tensor([3.7433, 3.4454, 3.4477, 3.2503, 3.0781, 3.9519, 3.2439, 2.9247, 2.9741,
         3.6923, 3.4820, 3.7436, 3.07

In [91]:
predictions = list(round(float(i), 1) for i in torch.cat(predictions))

In [92]:
val_data = pd.read_csv('../../data/dev_rtt.csv')

In [93]:
labels = val_data['label'].tolist()
len(labels)

197

In [100]:
cnt = 0
for idx, (pred, y) in enumerate(zip(predictions, labels)):
    if abs(pred - y) >= 0.5:
        cnt += 1
        print(f"model pred: {pred}")
        print(val_data.iloc[idx])
        print('-------------------------------------')

model pred: 2.7
id              boostcamp-sts-v1-dev-001
source                         slack-rtt
sentence_1                   감격스러워 입막으심?
sentence_2                너무 감동해서 입 다물어?
label                                3.4
binary-label                           1
Name: 0, dtype: object
-------------------------------------
model pred: 3.4
id                   boostcamp-sts-v1-dev-002
source                               nsmc-rtt
sentence_1      이번 년도에 본 영화 중 가장 최악의 영화......
sentence_2                    올해 본 영화 중 최악...
label                                     4.0
binary-label                                1
Name: 1, dtype: object
-------------------------------------
model pred: 3.7
id                  boostcamp-sts-v1-dev-008
source                             slack-rtt
sentence_1      고수님들의 많은 가르침 부탁드립니다 <PERSON>
sentence_2       고수님들의 많은 지도 부탁드립니다 <PERSON>
label                                    4.6
binary-label                               1
Name: 4, dtype: object
---------------

In [101]:
cnt

127

In [19]:
tokenizer.decode(dataloader.test_dataset.__getitem__(4)[0])

'[CLS] [ORG] 다음 밥스테이지가 기대됩니다 ~ ㅎ [SEP] 다음 후기도 기대됩니다 ~ ~ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [17]:
tokenizer(['[ORG]'])

{'input_ids': [[2, 35001, 3]], 'token_type_ids': [[0, 0, 0]], 'attention_mask': [[1, 1, 1]]}